# Colorado I-25 Traffic Data



![I-25](images\i-25.jpg)

In [ ]:
### This notebook scrapes hourly traffic count data from continuous
### recording traffic counters along I-25 in Colorado from the
### Colorado Dept. of Transportation (CDOT) Online Transportation
### Information System (OTIS).

### The collected data are written to the table [Traffic] within a sqlite 
### database stored at /data/traffic.db

### The table is in the format:
### Station_ID TEXT, Count_Date DATE, Direction TEXT,
### t00h INT - ... - t23H INT

### As written, the tool collects three years of daily data for ten
### stations, yielding 1095 continuous days of data at an hourly interval
### for each of tenn stations.

In [ ]:
import os
import sys
import time
import sqlite3

from urllib import urlopen
from bs4 import BeautifulSoup
from collections import defaultdict

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
### Functions

def get_data(url, stations, years, months):  # s,y,m = lists
    data = defaultdict(list)
    
    def get_rows(soup):
        rows = soup.find_all('tr')
        return [[td.text for td in row.find_all('td')] for row in rows]

    for station in stations:
        for year in years:
            for month in months:
                print 'Processing [{}]: {}/{}'.format(station, month, year)
                try:
                    html = urlopen(url.format(station, year, month))
                except:
                    print 'error opening url'
                try:
                    soup = BeautifulSoup(html, 'html.parser')
                    data[station].extend(get_rows(soup))
                except:
                    print 'error reading html'
                time.sleep(5)
    return data


def build_database(db_path):
    with sqlite3.connect(db_path) as con:
        cur = con.cursor()
        create_sql = ('CREATE TABLE Traffic('
                      'Station_ID TEXT, Count_Date DATE, Direction TEXT, '
                      't00h INT, t01h INT, t02h INT, t03h INT, t04h INT, '
                      't05h INT, t06h INT, t07h INT, t08h INT, t09h INT, '
                      't10h INT, t11h INT, t12h INT, t13h INT, t14h INT, '
                      't15h INT, t16h INT, t17h INT, t18h INT, t19h INT, '
                      't20h INT, t21h INT, t22h INT, t23h INT);')
        cur.execute(create_sql)

        
def insert_rows(db_path, rows):  # db path, list of tuples
    with sqlite3.connect(db_path) as con:
        cur = con.cursor()
        insert_sql = ('INSERT INTO Traffic '
                      'VALUES('
                      '?,?,?,'
                      '?,?,?,?,?,?,?,?,'
                      '?,?,?,?,?,?,?,?,'
                      '?,?,?,?,?,?,?,?);')
        cur.executemany(insert_sql, rows)
        

def main_get_traffic_data(url, stations, years, months, db_path):
    traffic_data = get_data(url, stations, years, months)
    
    if not os.path.exists(db_path):
        build_database(db_path)
        
    for station, data in traffic_data.items():
        in_rows = [tuple([station] + row) for row in data if row]
        print 'Adding records for station [{}]'.format(station)
        insert_rows(traffic_db, in_rows)

In [ ]:
### Input parameters

traffic_db = r'C:\Users\michael\Documents\projects\traffic\data\traffic.db'

# url to format: station ID, year, month
url = (r'http://dtdapps.coloradodot.info/otis/TrafficData'
       r'/GetDailyTrafficVolumeForStationByMonth/{}/true/{}/{}')

# continuous recording stations
stations = ['000101', '100950', '000009', '000520', '000124',
            '000012', '000501', '000103', '000106', '000127',]

# period of interst
years, months = [2015, 2016, 2017], range(1, 13)

In [ ]:
main_get_traffic_data(url, stations, years, months, traffic_db)
# main_get_traffic_data(url, ['000101'], [2017], [1], traffic_db)

In [ ]:
db = sqlite3.connect(traffic_db)
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", db)

In [ ]:
with sqlite3.connect(traffic_db) as con:
    traffic_df = pd.read_sql_query("SELECT * FROM Traffic;", con)
traffic_df.info()
# traffic_df.head()